In [1]:
import re
import unicodedata
import pandas as pd
import nltk
import prepare
import acquire
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from env import user, password, host

def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'
from wordcloud import WordCloud

url = get_db_url("spam_db")
sql = "SELECT * FROM spam"

df = pd.read_sql(sql, url, index_col="id")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samkeeler/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samkeeler/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Viewing the spread of spam vs. non-spam

df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [3]:
# Using prepare.py file function to clean and lemmatize text data. Removing unnecessary columns from the created
# dataframe and renaming the remaining column to 'text'

df = prepare.make_prepped_columns(df, 'text')
df.drop(columns = ['text', 'clean', 'stemmed'], inplace = True)
df.rename(columns = {'lemmatized': 'text'}, inplace = True)

In [4]:
df

,label,text
id,,
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don ' t think he go to usf he life aroun...
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...
5568,ham,will i b going to esplanade fr home
5569,ham,pity wa in mood for that soany other suggestion


In [5]:
# Creating a function to split the data into train, validate, test

def split(df, stratify_by=None):
    train, test = train_test_split(df, test_size=.2, random_state=123, stratify=df[stratify_by])
    
    train, validate = train_test_split(train, test_size=.3, random_state=123, stratify=train[stratify_by])
    
    return train, validate, test

In [6]:
# Splitting the data

train, validate, test = split(df, stratify_by='label')

In [7]:
# Specifying the data to model off of

X_train = train.text
X_validate = validate.text
X_test = test.text

In [8]:
# Specifying the labels that I am trying to predict

y_train = train.label
y_validate = validate.label
y_test = test.label

In [9]:
# Creating a vectorizer object 

tfidf = TfidfVectorizer()

# Fitting that object onto the train data

tfidf.fit(X_train)

# Applying the vector transformer to each data set

X_train_vectorized = tfidf.transform(X_train)
X_validate_vectorized = tfidf.transform(X_validate)
X_test_vectorized = tfidf.transform(X_test)

In [10]:
# Creating and fitting the modeling object

lm = LogisticRegression()
lm.fit(X_train_vectorized, y_train)

LogisticRegression()

In [11]:
# Creating a dataframe that will hold predicted and actual values for evaluation metrics

train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))

In [12]:
# Applying the modeling object to predict

train['predicted'] = lm.predict(X_train_vectorized)
validate["predicted"] = lm.predict(X_validate_vectorized)
test['predicted'] = lm.predict(X_test_vectorized)

In [13]:
# Train accuracy

(train.actual == train.predicted).mean()

0.9653735171529336

In [14]:
# Validate accuracy

(validate.actual == validate.predicted).mean()

0.9581464872944694

In [15]:
# Test accuracy

(test.actual == test.predicted).mean()

0.9632286995515695

## Breaking Bad

In [16]:
def acquire_script():
    script = pd.read_csv('breaking_bad_dataset.csv')
    script.drop(columns = ['Unnamed: 0'], inplace = True)
    return script

In [17]:
script = acquire_script()
script.head()

,Text,Speaker,Season
0,[in a calm tone] - Then what do you suggest w...,Walter,4
1,Look you remember Emilio? Kay this dude got E...,Jesse,5
2,You are joking right? If I gave you that mone...,Walter,3
3,Yeah damn straight. Know why? Cause we do thi...,Jesse,5
4,Stay out of my territory.,Walter,3


In [18]:
def prep_script(script):
    script = script.replace(r'\[.*?\]', '',regex=True)
    script['Text'] = script['Text'].apply(lambda x: str(x))
    script = prepare.make_prepped_columns(script, 'Text')
    script.drop(columns = ['Text', 'clean', 'stemmed'], inplace = True)
    script.rename(columns = {'lemmatized': 'text'}, inplace = True)
    return script

In [19]:
script = prep_script(script)
script.head()

,Speaker,Season,text
0,Walter,4,then what do you suggest we do i dont think ei...
1,Jesse,5,look you remember emilio kay this dude got emi...
2,Walter,3,you are joking right if i gave you that money ...
3,Jesse,5,yeah damn straight know why cause we do thing ...
4,Walter,3,stay out of my territory


In [20]:
script.Speaker.value_counts()

Walter    719
Hank      558
Jesse     558
Skyler     79
Marie       5
Name: Speaker, dtype: int64

In [23]:
train, validate, test = split(script, stratify_by = 'Speaker')

In [25]:
X_train = train.text
X_validate = validate.text
X_test = test.text

In [26]:
y_train = train.Speaker
y_validate = validate.Speaker
y_test = test.Speaker

In [27]:
# Creating a vectorizer object 

tfidf = TfidfVectorizer()

# Fitting that object onto the train data

tfidf.fit(X_train)

# Applying the vector transformer to each data set

X_train_vectorized = tfidf.transform(X_train)
X_validate_vectorized = tfidf.transform(X_validate)
X_test_vectorized = tfidf.transform(X_test)

In [28]:
# Creating and fitting the modeling object

lm = LogisticRegression()
lm.fit(X_train_vectorized, y_train)

LogisticRegression()

In [29]:
# Creating a dataframe that will hold predicted and actual values for evaluation metrics

train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))

In [30]:
# Applying the modeling object to predict

train['predicted'] = lm.predict(X_train_vectorized)
validate["predicted"] = lm.predict(X_validate_vectorized)
test['predicted'] = lm.predict(X_test_vectorized)

In [31]:
# Train accuracy

(train.actual == train.predicted).mean()

0.7318435754189944

In [33]:
pd.concat([train, X_train], axis = 1)

,actual,predicted,text
565,Hank,Jesse,look i know some guy alright i can create a ne...
1579,Hank,Hank,okay so they got name
1378,Hank,Jesse,yo im really sorry okay
1050,Walter,Walter,it is impeccable it is the purest most chemica...
1480,Hank,Hank,i remember oh i remember that you cook i sell ...
...,...,...,...
1520,Walter,Walter,12 million
1217,Hank,Jesse,yeah thats the thing yknow your scumbag brothe...
958,Hank,Hank,oh we are dead dead men muerto or muerte or ho...
1538,Hank,Hank,what are we building


In [37]:
X_train

565     look i know some guy alright i can create a ne...
1579                                okay so they got name
1378                              yo im really sorry okay
1050    it is impeccable it is the purest most chemica...
1480    i remember oh i remember that you cook i sell ...
                              ...                        
1520                                           12 million
1217    yeah thats the thing yknow your scumbag brothe...
958     oh we are dead dead men muerto or muerte or ho...
1538                                 what are we building
609     when skyler i wa medicated i mean i could have...
Name: text, Length: 1074, dtype: object

In [39]:
train.actual.value_counts()

Walter    402
Jesse     313
Hank      312
Skyler     44
Marie       3
Name: actual, dtype: int64

In [41]:
script.loc[1217]

Speaker                                                 Hank
Season                                                     5
text       yeah thats the thing yknow your scumbag brothe...
Name: 1217, dtype: object

In [ ]:
# After reviewing I found this data set is not accurate and speakers aren't always correct:(

## Article predictor

In [43]:
articles = acquire.get_all_inshorts(["business", "sports", "technology", "entertainment", "science", "world"])